In [1]:
#%pip install langchain langchain-ollama langchain-text-splitters langchain-chroma langchain-community gradio chromadb pypdf

In [2]:
#importamos todo lo que vamos a necesitar

import os
import logging
import time
import sys
import tempfile
from typing import List, Dict, Any

from langchain_chroma import Chroma
from langchain_ollama import ChatOllama, OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain_core.prompts import PromptTemplate

import gradio as gr

/Users/luciaderulos/.pyenv/versions/3.11.7/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Configuración del logging
logging.basicConfig(    
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    )
logger = logging.getLogger(__name__)

In [4]:
# Configuramos los parámetros de Ollama

PERSISTENCE_DIR = "chroma_db"
CHUNK_SIZE = 500
CHUNK_OVERLAP = 100
PDF_URLS = [
    "https://www.sanidad.gob.es/areas/saludDigital/doc/eIASNS_v13.pdf",
    "https://www.unespa.es/main-files/uploads/2020/01/El-seguro-del-hogar-y-los-gremios-de-reparadores-FINAL.pdf",

]

LLM_MODEL = "gemma3:270m"  # Reemplaza con el nombre de tu modelo Ollama
EMBEDDING_MODEL = "embeddinggemma:300m"  # Reemplaza con el nombre de tu modelo de embeddings Ollama
TEMPERATURE = 0.1 #temperatura baja, determinista

In [5]:
#clase rag 

class RAGSystem:
    
    def __init__(self, pdf_urls: List[str], persistence_dir: str = PERSISTENCE_DIR):
        
        self.pdf_urls = pdf_urls
        self.persistence_dir = persistence_dir
        self.documents = []
        self.vector_store = None
        self.llm = None
        self.chain = None

        callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
        self.llm = ChatOllama(
            model=LLM_MODEL,
            temperature=TEMPERATURE,
            callback_manager=callback_manager            
        )
        
        self.embeddings = OllamaEmbeddings(model=EMBEDDING_MODEL)        
        logger.info(f"Initialized RAG system with {len(pdf_urls)} PDFs")
    

In [6]:
def load_documents(self):
    logger.info("Loading documents from PDFs...")

    text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=CHUNK_SIZE,
            chunk_overlap=CHUNK_OVERLAP,
            separators=["\n\n", "\n", " ", "", "."]
    )

    all_pages = []
    for url in self.pdf_urls:
        try:
            loader = PyPDFLoader(url)
            pages = loader.load()
            logger.info(f"Loaded {len(pages)} pages from {url}")
            all_pages.extend(pages)
        except Exception as e:
            logger.error(f"Error loading {url}: {e}")

    # divimos en chunks        
    self.documents = text_splitter.split_documents(all_pages)
    logger.info(f"Split documents into {len(self.documents)} chunks")

In [7]:
def create_vectorstore(self) -> None:
    
   #borramos la carpeta de persistencia si existe 
    if os.path.exists(self.persistence_dir):
        import shutil
        logger.info(f"Removing existing persistence directory at {self.persistence_dir}")
        shutil.rmtree(self.persistence_dir, ignore_errors=True)


    # Creamos el vector store
    logger.info("Creating vector store...")
    if not self.documents:
      self.load_documents()

    #creamos la carpeta de persistencia si no existe
    if not os.path.exists(self.persistence_dir):
        os.makedirs(self.persistence_dir)

    #creamos el vector store con manejo de errores
    try:
        self.vector_store = Chroma.from_documents(
            documents=self.documents,
            embedding=self.embeddings,
            persist_directory=self.persistence_dir
        )

        
        logger.info(f"Vectorstore creada satisfactoriamente con {len(self.documents)} documentos")
    except Exception as e:
        logger.error(f"Error creating vector store: {e}")
        raise
         

In [8]:
def setup_chain(self):

    if not self.vector_store:
        self.create_vectorstore()

    #creamos el retriever
    retriever = self.vector_store.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 3}
    )
    # Definimos el prompt template
    template = """
    Eres un asistente útil que ayuda a responder preguntas basadas en los siguientes documentos proporcionados de forma educada y profesional.
    Utiliza la información de los documentos para formular una respuesta precisa y completa.               
    
    (1) Se atento al detalle y asegúrate de que la respuesta sea relevante para la pregunta realizada.
    (2) Si la información no está disponible en los documentos, responde informando que los documentos no contienen la información solicitada.
    (3) Si el contexto lo permite, responde de forma detalla y precisa.
    (4) No inventes información ni hagas suposiciones; limita tus respuestas únicamente a la información proporcionada en los documentos.
    (5) Revisa tu respuesta antes de enviarla para asegurarte de que sea clara y coherente, y responda a la pregunta realizada.
    (6) Debajo de la respuesta, incluye una sección titulada "Fuentes" donde enumeres las fuentes de la información utilizada en la respuesta.

    Piensa paso a paso y proporciona la respuesta en español.

    Utiliza el siguiente formato para estructurar tu respuesta:
    ### Pregunta: {question}###
    ### Respuesta: {context} ###
    ### Fuentes:  ###
     
    """  

    prompt = PromptTemplate.from_template(template)

    # Configuramos la cadena de RAG
    self.chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | self.llm
        | StrOutputParser()
    )

    logger.info("RAG cadena configurada correctamente.")

In [9]:
# Método para responder preguntas
def answer_question(self, question: str) -> str:

    if not self.chain:
        self.setup_chain()

    logger.info(f"Respondiendo pregunta: {question}")   
    try:
        answer = self.chain.invoke(question)
        return answer
    except Exception as e:
        logger.error(f"Error answering question: {e}")
        return "Lo siento, ha ocurrido un error al procesar tu pregunta."


In [10]:
#interfaz gradio
def create_gradio_interface(rag_system: RAGSystem) -> gr.Interface:
   
    def get_answer(question: str) -> str:
        return rag_system.answer_question(question)
    
    # Definimos la interfaz de Gradio
    interface = gr.Interface(
        fn=get_answer,
        inputs=gr.Textbox(
            label="Tu Pregunta",
            placeholder="Escribe tu pregunta aquí..."),
        outputs=gr.Markdown(label="Respuesta"),
        title="Sistema RAG con Ollama y LangChain",
        description="Haz preguntas basadas en los documentos PDF cargados.",
        theme=gr.themes.Soft(),

        # ejemplos de preguntas
        examples=[
            "¿Cuáles son las principales características de la IA en sanidad?",
            "¿Qué recomendaciones se dan para el seguro del hogar?",
            "¿Cómo afecta la digitalización a los servicios de salud?",
            "¿Qué medidas de seguridad se sugieren para la protección de datos en sanidad?"
        ]
    )
    return interface
    
   

In [11]:
def main() -> None:

    try:

        print("Comprobando Ollama Models disponibles...")
        try:
            import requests
            response = requests.get("http://localhost:11434/api/tags")
            print("Ollama Models disponibles:")
            if response.status_code == 200:
                for model in response.json().get("models", []):
                    print(f"- {model['name']}")
            else:
                print(f"Error al obtener los modelos: {response.status_code}")
        except Exception as e:
            print(f"No se pudo conectar a Ollama API: {e}")

        print(f"Usando modelo: {LLM_MODEL}")
        print(f"y modelo de embedding: {EMBEDDING_MODEL}")
        print("Inicializando sistema RAG...")

        #creamos el sistema RAG
        rag_system = RAGSystem(pdf_urls=PDF_URLS)

        #cargamos los documentos y creamos el vector store
        rag_system.load_documents()
        rag_system.create_vectorstore()

        #test con pregunta de control
        logger.info("Realizando pregunta de control...")
        test_answer= rag_system.answer_question("¿Cuáles son las principales características de la IA en sanidad?")
        logger.info(f"Respuesta de control recibida (tamaño: {len(test_answer)})")

        #creamos la interfaz de gradio
        logger.info("Creando interfaz de Gradio...")
        gr_interface = create_gradio_interface(rag_system)
        gr_interface.launch(share=False)

    except Exception as e:
        logger.error(f"Error en la ejecución principal: {e}")
     


In [12]:
# añadir los métodos a la clase RAGSystem
RAGSystem.load_documents = load_documents
RAGSystem.create_vectorstore = create_vectorstore   
RAGSystem.setup_chain = setup_chain
RAGSystem.answer_question = answer_question

In [ ]:
# ejecutamos el main
if __name__ == "__main__":
    main()


Comprobando Ollama Models disponibles...
Ollama Models disponibles:
- gemma3:270m
- embeddinggemma:300m
- gemma3:1b
Usando modelo: gemma3:270m
y modelo de embedding: embeddinggemma:300m
Inicializando sistema RAG...


2026-01-21 21:49:08,244 - __main__ - INFO - Initialized RAG system with 2 PDFs
2026-01-21 21:49:08,246 - __main__ - INFO - Loading documents from PDFs...
2026-01-21 21:49:23,921 - __main__ - INFO - Loaded 45 pages from https://www.sanidad.gob.es/areas/saludDigital/doc/eIASNS_v13.pdf
2026-01-21 21:49:28,486 - __main__ - INFO - Loaded 22 pages from https://www.unespa.es/main-files/uploads/2020/01/El-seguro-del-hogar-y-los-gremios-de-reparadores-FINAL.pdf
2026-01-21 21:49:28,512 - __main__ - INFO - Split documents into 376 chunks
2026-01-21 21:49:28,515 - __main__ - INFO - Removing existing persistence directory at chroma_db
2026-01-21 21:49:28,518 - __main__ - INFO - Creating vector store...
2026-01-21 21:49:28,587 - chromadb.telemetry.product.posthog - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2026-01-21 21:55:28,768 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2026-01

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


2026-01-21 21:56:29,067 - httpx - INFO - HTTP Request: HEAD https://huggingface.co/api/telemetry/https%3A/api.gradio.app/gradio-launched-telemetry "HTTP/1.1 200 OK"
2026-01-21 21:57:02,175 - __main__ - INFO - Respondiendo pregunta: que sabes sobre la seguridad y la ia 
2026-01-21 21:57:02,993 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2026-01-21 21:57:12,299 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2026-01-21 21:57:57,947 - __main__ - INFO - Respondiendo pregunta: dame una receta de tarta de manzana 
2026-01-21 21:57:58,534 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
2026-01-21 21:58:08,963 - httpx - INFO - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2026-01-21 21:58:29,675 - __main__ - INFO - Respondiendo pregunta: ¿Qué recomendaciones se dan para el seguro del hogar?
2026-01-21 21:58:30,170 - httpx - INFO - HTTP Request: POST htt